In [ ]:
'''
!!!!!!! Эта ячейка для запуска дедубликатора на Google Drive через Google Colab !!!!!!!

'''

from google.colab import drive
drive.mount('/content/drive')
# Устанавливаем рабочую директорию
pic_path = "/content/drive/My Drive/UII/Capture"
# Создаем директорию для дубликатов
dup_path = "/content/drive/My Drive/UII/Capture/Duplicates"

In [2]:
'''
!!!!!!! Эта ячейка для запуска дедубликатора на локальной машине через Jupyter Notebook (или IDE) !!!!!!!

'''

# Устанавливаем рабочую директорию
pic_path = r'C:\Users\anton\UII\AZavod_Ural'
# Создаем директорию для дубликатов
dup_path = r'C:\Users\anton\UII\AZavod_Ural\Duplicates'

In [9]:
'''
!!!!!!! Собственно сам дедубликатор !!!!!!!

'''

import os
import cv2
import random
from PIL import Image
import numpy as np
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from concurrent.futures import ThreadPoolExecutor
import math
import gc
from collections import defaultdict
import concurrent.futures

os.makedirs(dup_path, exist_ok=True)
   
# Функция для вычисления структурного хэша изображения с динамической размерностью, в зависимости от размера изображения
def dhash(image, base_size=64):
    initial_hash_size = max(image.shape[:2]) / base_size # Вычисляем начальный размер хэша
    hash_size = 2 ** math.ceil(math.log2(initial_hash_size)) # Округляем до ближайшей степени двойки
    hash_size = max(4, hash_size) # Ограничиваем минимальный размер хэша
    # Вычисляем хэш изображения
    resized = cv2.resize(image, (hash_size + 1, hash_size))
    diff = resized[:, 1:] > resized[:, :-1]
    return diff.flatten().astype(int)  # Возвращаем хэш в виде вектора битов

# Функция для вычисления цветового хэша изображения
def color_hash(image, bins=8):
    # Конвертируем изображение в HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Вычисляем гистограмму для каждого канала HSV
    hist_h = cv2.calcHist([hsv_img], [0], None, [bins], [0, 180])
    hist_s = cv2.calcHist([hsv_img], [1], None, [bins], [0, 256])
    hist_v = cv2.calcHist([hsv_img], [2], None, [bins], [0, 256])
    # Нормализуем гистограмму
    cv2.normalize(hist_h, hist_h)
    cv2.normalize(hist_s, hist_s)
    cv2.normalize(hist_v, hist_v)
    # Объединяем гистограммы в один вектор
    return np.concatenate((hist_h, hist_s, hist_v)).flatten()

# Функция для сравнения двух цветовых хэшей
def compare_color_hashes(hash1, hash2):
    # Вычисляем Евклидово расстояние между двумя хэшами
    distance = np.linalg.norm(hash1 - hash2)
    return distance    

# Функция для загрузки и хэширования изображений      
def load_and_hash_image(path):
    try:
        # Загружаем изображение с помощью Pillow
        with Image.open(path) as img:
            # Конвертируем изображение в формат BGR для OpenCV                
            bgr_image = np.array(img.convert('RGB'))[:, :, ::-1]
        dhash_value = dhash(bgr_image)  # Вычисляем структурный хэш
        color_hash_value = color_hash(bgr_image)  # Вычисляем цветовой хэш
        return (path, (dhash_value, color_hash_value, bgr_image))
    except IOError:
        print(f"Не удалось загрузить изображение: {path}")
        return (path, (None, None, None))        

# Функция хэширования изображений в многопоточном режиме
def process_images(paths):
    hash_dict = {}
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_and_hash_image, paths))
    for path, (dhash_value, color_hash_value, original_image) in results:
        if dhash_value is not None:
            hash_dict[path] = (dhash_value, color_hash_value, original_image)
    gc.collect() # очистка памяти
    return hash_dict

# Функция для определения медианного размера изображений
def median_image_size(images):
    sizes = [Image.open(img).size for img in images]
    widths, heights = zip(*sizes)
    median_width = sorted(widths)[len(widths) // 2]
    median_height = sorted(heights)[len(heights) // 2]
    median_size = (median_width, median_height)
    return median_size

# Функция для изменения размера изображения до медианного размера
def resize_to_median(image, median_size):
    return cv2.resize(image, median_size, interpolation=cv2.INTER_AREA)    

# Функция хэширования изображений с приведенными размерами в многопоточном режиме
def process_images_resize_mode(paths, median_size):
    def process_single_image(path):
        try:
            with Image.open(path) as img:
                bgr_image = np.array(img.convert('RGB'))[:, :, ::-1]
                resized_image = resize_to_median(bgr_image, median_size)
            dhash_value = dhash(resized_image)
            color_hash_value = color_hash(resized_image)
            return (path, (dhash_value, color_hash_value, bgr_image))
        except IOError:
            print(f"Не удалось загрузить изображение: {path}")
            return (path, (None, None, None))

    hash_dict = {}
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_single_image, path) for path in paths]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(paths), desc="Предобработка и хэширование", unit=" файлов"):
            path, data = future.result()
            if data[0] is not None:
                hash_dict[path] = data
    gc.collect() # очистка памяти
    return hash_dict

# Создаем KD-дерево для быстрого поиска близких хэшей
def create_kd_tree(hash_dict):
    hash_values = [val[0] for val in hash_dict.values()]
    tree = KDTree(hash_values)
    return tree

# Функция для проверки схожести двух изображений по структурному и цветовому хэшам
def is_similar(hash1, hash2, color_hash1, color_hash2, similar_threshold):
    structural_similarity = (1 - np.sum(hash1 != hash2) / len(hash1)) * 100
    color_similarity = (1 - compare_color_hashes(color_hash1, color_hash2)) * 100
    total_similarity = (structural_similarity + color_similarity) / 2
    return total_similarity >= similar_threshold

# Функция поиска дубликатов по KD-дереву  
def find_duplicates(hash_dict, kd_tree, similar_threshold, show_progress):
    duplicate_groups = {}
    iterable = tqdm(enumerate(hash_dict.items()), total=len(hash_dict), desc="Поиск дубликатов", unit=" изображений") if show_progress else enumerate(hash_dict.items())
    for idx, (file, (dhash_value, color_hash_value, _)) in iterable:
        similar_indices = kd_tree.query_ball_point(dhash_value, similar_threshold)
        duplicate_group = [file]
        for idx in similar_indices:
            similar_file = list(hash_dict.keys())[idx]
            if file != similar_file:
                similar_dhash, similar_color_hash, _ = hash_dict[similar_file]
                if is_similar(dhash_value, similar_dhash, color_hash_value, similar_color_hash, similar_threshold):
                    duplicate_group.append(similar_file)
        if len(duplicate_group) > 1:
            duplicate_groups[len(duplicate_groups)] = list(set(duplicate_group))
    return duplicate_groups

# Функция для поиска дубликатов между группами для их объединения по пересекающимся элементам
def merge_duplicate_groups(duplicate_groups):
    merged_groups = []
    for group in duplicate_groups.values():
        # Ищем существующую группу, с которой есть пересечение
        found = False
        for merged in merged_groups:
            if set(group).intersection(merged):
                merged.update(group)
                found = True
                break
        if not found:
            merged_groups.append(set(group))
    return merged_groups

# Функция для отображения изображений
def display_images(images):
    n_images = len(images)
    plt.figure(figsize=(5 * n_images, 5))  # Изменяем размер фигуры в зависимости от количества изображений
    for i, (title, image) in enumerate(images.items()):
        ax = plt.subplot(1, n_images, i + 1)  # Создаем подграфик для каждого изображения
        ax.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        ax.axis('off')
        ax.set_title(title)
    plt.show()

# Функция для группировки изображений по размерам
def group_images_by_size(directory):
    size_groups = defaultdict(list)
    for file in tqdm(os.listdir(directory), desc="Группировка по размеру", unit=" файлов"):
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
            path = os.path.join(directory, file)
            try:
                image = Image.open(path)
                size = image.size
            except IOError:
                print(f"Failed to load image: {file}")
                continue
            if image is not None:
                size_groups[size].append(path)
    return size_groups.values()

# Функция для дедубликации и перемещения дубликатов
def deduplicate_and_move(hash_dict, similar_threshold, show_dups, resize_mode):
    if len(hash_dict) > 1:
        kd_tree = create_kd_tree(hash_dict)
        duplicate_groups = find_duplicates(hash_dict, kd_tree, similar_threshold, resize_mode)
        merged_duplicate_groups = merge_duplicate_groups(duplicate_groups)

        for group in merged_duplicate_groups:    
            if len(group) > 1:
                original = random.choice(list(group))
                duplicates = group - {original}
                if show_dups:
                    images_to_show = {f"Original": hash_dict[original][2]}
                    for i, dup in enumerate(duplicates):
                        images_to_show[f"Duplicate {i+1}"] = hash_dict[dup][2]
                    display_images(images_to_show)
                for dup in duplicates:
                    dup_filename = os.path.basename(dup)
                    target_path = os.path.join(dup_path, dup_filename)
                    if not os.path.exists(target_path):
                        shutil.move(dup, target_path)

# Основная функция для дедубликации с учетом режима ресайза
def dedublicator(directory, similar_threshold, show_dups, resize_mode):
    if resize_mode:
        all_images = [os.path.join(directory, file) for file in os.listdir(directory) if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'))]
        median_size = median_image_size(all_images)
        hash_dict = process_images_resize_mode(all_images, median_size)
        deduplicate_and_move(hash_dict, similar_threshold, show_dups, resize_mode)
    else:
        image_groups = group_images_by_size(directory)
        for group in tqdm(image_groups, desc="Поиск дубликатов в группах", unit=" групп"):
            hash_dict = process_images(group)
            deduplicate_and_move(hash_dict, similar_threshold, show_dups, resize_mode)                        

#============================  Основное тело  ============================#

# Параметры

similar_threshold = 90 # степень похожести кадров (в %)
show_dups = False # выводить ли на экран оригинал с дубликатами
resize_mode = True  # Установите True для обработки любых размеров и False для поиска дубликатов в совпадающих размерах изображений
   
dedublicator(pic_path, similar_threshold, show_dups, resize_mode)
gc.collect() # очистка памяти

cnt_img_org = len(os.listdir(pic_path))  # Подсчет количества изображений в исходной папке
cnt_img_dup = len(os.listdir(dup_path)) # Подсчет количества перемещенных изображений
print(f"Похожие изображения успешно обработаны. Из {cnt_img_org+cnt_img_dup} файлов удалено {cnt_img_dup} похожих и дубликатов. Осталось {cnt_img_org} разных изображений")

Поиск дубликатов: 100%|██████████| 1351/1351 [01:20<00:00, 16.79 изображений/s]


Похожие изображения успешно обработаны. Из 1364 файлов удалено 289 похожих и дубликатов. Осталось 1075 разных изображений
